In [1]:
import itertools
from parlay_system import *
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)


In [2]:
# Dec 15 games

# # NOTE: Odds boosted to +105 for a patriots, chiefs, sf49ers outcome
# buccs = MoneyLine(event="buccs", bet_amount=100, odds=143)
# texans = MoneyLine(event="texans", bet_amount=100, odds=-165)

# bills = MoneyLine(event="bills", bet_amount=100, odds=245)
# patriots = MoneyLine(event="patriots", bet_amount=100, odds=-286)

# rams = MoneyLine(event="rams", bet_amount=100, odds=235)
# sf49ers = MoneyLine(event="sf49ers", bet_amount=100, odds=-275)

# binaries = [
#     [buccs, texans],
#     [bills, patriots],
#     [rams, sf49ers]
# ]

# results = [
#     [0, 1], #[patriots, bengals],
#     [0, 1], #[broncos, chiefs],
#     [0, 1], #[falcons, sf49ers],
# ]

# NOTE: Odds boosted to +105 for a patriots, chiefs, sf49ers outcome
delaware = MoneyLine(event="delaware", bet_amount=100, odds=-200)
liu = MoneyLine(event="liu", bet_amount=100, odds=165)

nalabama = MoneyLine(event="nalabama", bet_amount=100, odds=-225)
alabamaam = MoneyLine(event="alabamaam", bet_amount=100, odds=180)

ucirvine = MoneyLine(event="ucirvine", bet_amount=100, odds=-245)
uic = MoneyLine(event="uic", bet_amount=100, odds=200)

binaries = [
    [delaware, liu],
    [nalabama, alabamaam],
    [ucirvine, uic]
]

results = [
    [0, 1], #[towson, liberty],
    [0, 1], #[ucirvine, illchi],
    [0, 1], #[wizards, sf49ers],
]

override_arr = [0, 2, 5]


def create_dicts(in_binaries, in_results):
    ml_lookup = {}
    ml_win_lookup = {}
    id = 0
    for bi in range(len(in_binaries)):
        binary = in_binaries[bi]
        for ml_i in range(len(binary)):
            ml = binary[ml_i]
            ml.set_index(id) 
            is_winner = in_results[bi][ml_i]
            ml_lookup[id] = ml.event
            ml_win_lookup[id] = is_winner
            id += 1
    
    return ml_lookup, ml_win_lookup

ml_lookup, ml_outcome_lookup = create_dicts(binaries, results)

In [3]:
def findsubsets(s, n):
    return list(itertools.combinations(s, n))

TOURNEY_SIZE = len(binaries)
SELECT_NUM = 3
s = set()
n = SELECT_NUM

for i in range(TOURNEY_SIZE):
    s.add(i)

all_subsets = findsubsets(s, n)

In [4]:
len(all_subsets)

1

In [5]:
csv = []
df = pd.DataFrame()

for i in range(len(all_subsets)):
    current_subset = all_subsets[i]
    parlay_binaries = []
    for i in range(len(current_subset)):
        index = current_subset[i]
        parlay_binaries.append(binaries[index])

    ps = ParlaySystem(binaries=parlay_binaries,
                    target_profit=0.15,
                    bounds=(1, 30),
                    binary_index_arr=list(current_subset),
                    binary_results_arr=results,
                    index_to_ml=ml_lookup,
                    index_to_outcome=ml_outcome_lookup,
                    override_arr=override_arr
                    )
    
    csv_res, raw_df = ps.slsqp_solver()
    df = pd.concat([df, raw_df])
    csv.append(csv_res)

yooooo delaware_nalabama_uic [0, 2, 5] 663 7.63
     fun: -1.852907315127149
     jac: array([ 0.49362245, -0.07875   ,  0.13571428, -0.7       ,  0.201233  ,
       -0.56041667, -0.43107143, -1.9075    ])
 message: 'Positive directional derivative for linesearch'
    nfev: 180
     nit: 14
    njev: 10
  status: 8
 success: False
       x: array([2.79354816, 1.59688601, 1.828194  , 1.        , 1.92822297,
       1.09724238, 1.20590648, 1.        ])
slsqp_solver: 
                         event    index[]     result  event_status         odds    bet       mult   payout  profit
7            liu_alabamaam_uic  [1, 3, 5]  [1, 1, 1]          True  2126.000000  1.000  22.260000  22.2600   9.810
3       delaware_alabamaam_uic  [0, 3, 5]  [0, 1, 1]         False  1160.000000  1.000  12.600000  12.6000   0.150
5             liu_nalabama_uic  [1, 2, 5]  [1, 0, 1]         False  1048.333333  1.097  11.483333  12.6000   0.150
6       liu_alabamaam_ucirvine  [1, 3, 4]  [1, 1, 0]         False   94

In [6]:
df.describe()

,odds,bet,mult,payout,profit
count,8.000000,8.000000,8.000000,8.000000,8.000000
mean,884.716837,1.556250,9.847168,12.746662,0.296500
std,598.965010,0.620846,5.989650,4.105762,4.105861
min,205.102041,1.000000,3.051020,8.523200,-3.927000
25%,478.324830,1.072750,5.783248,10.707725,-1.742750
50%,803.928571,1.401500,9.039286,12.392100,-0.058000
75%,1076.250000,1.853000,11.762500,12.600000,0.150000
max,2126.000000,2.794000,22.260000,22.260000,9.810000


In [7]:
negative_df = df.loc[df.profit < 0]
negative_df = negative_df.sort_values(by=['event_status', 'profit'], ascending=[False, True])
negative_df.describe()

,odds,bet,mult,payout,profit
count,4.000000,4.000000,4.000000,4.000000,4.000000
mean,449.636054,2.036750,5.496361,10.478325,-1.972000
std,189.015439,0.523517,1.890154,1.511226,1.511207
min,205.102041,1.597000,3.051020,8.523200,-3.927000
25%,380.535714,1.770250,4.805357,9.925850,-2.524500
50%,465.221088,1.878000,5.652211,10.602950,-1.847500
75%,534.321429,2.144500,6.343214,11.155425,-1.295000
max,663.000000,2.794000,7.630000,12.184200,-0.266000


In [8]:
positive_df = df.loc[df.profit >= 0]
positive_df = positive_df.sort_values(by=['event_status', 'profit'], ascending=[False, False])
positive_df.describe()

,odds,bet,mult,payout,profit
count,4.000000,4.000000,4.000000,4.000,4.000
mean,1319.797619,1.075750,14.197976,15.015,2.565
std,544.601009,0.098137,5.446010,4.830,4.830
min,944.857143,1.000000,10.448571,12.600,0.150
25%,1022.464286,1.000000,11.224643,12.600,0.150
50%,1104.166667,1.048500,12.041667,12.600,0.150
75%,1401.500000,1.124250,15.015000,15.015,2.565
max,2126.000000,1.206000,22.260000,22.260,9.810


In [9]:
negative_df.head(200)

,event,index[],result,event_status,odds,bet,mult,payout,profit
0,delaware_nalabama_ucirvine,"[0, 2, 4]","[0, 0, 0]",False,205.102041,2.794,3.051020,8.5232,-3.927
4,liu_nalabama_ucirvine,"[1, 2, 4]","[1, 0, 0]",False,439.013605,1.928,5.390136,10.3934,-2.057
2,delaware_alabamaam_ucirvine,"[0, 3, 4]","[0, 1, 0]",False,491.428571,1.828,5.914286,10.8125,-1.638
1,delaware_nalabama_uic,"[0, 2, 5]","[0, 0, 1]",False,663.000000,1.597,7.630000,12.1842,-0.266


In [10]:
positive_df.head(200)

,event,index[],result,event_status,odds,bet,mult,payout,profit
7,liu_alabamaam_uic,"[1, 3, 5]","[1, 1, 1]",True,2126.000000,1.000,22.260000,22.26,9.81
3,delaware_alabamaam_uic,"[0, 3, 5]","[0, 1, 1]",False,1160.000000,1.000,12.600000,12.60,0.15
5,liu_nalabama_uic,"[1, 2, 5]","[1, 0, 1]",False,1048.333333,1.097,11.483333,12.60,0.15
6,liu_alabamaam_ucirvine,"[1, 3, 4]","[1, 1, 0]",False,944.857143,1.206,10.448571,12.60,0.15


In [11]:
positive_df.groupby('event_status').count()

,event,index[],result,odds,bet,mult,payout,profit
event_status,,,,,,,,
False,3,3,3,3,3,3,3,3
True,1,1,1,1,1,1,1,1


In [12]:
negative_df.groupby('event_status').count()

,event,index[],result,odds,bet,mult,payout,profit
event_status,,,,,,,,
False,4,4,4,4,4,4,4,4


In [13]:
neg = negative_df.reset_index(drop=True).style.applymap(color_negative_red)
neg

,event,index[],result,event_status,odds,bet,mult,payout,profit
0,delaware_nalabama_ucirvine,"['0', '2', '4']","[0, 0, 0]",False,205.102,2.794,3.05102,8.5232,-3.927
1,liu_nalabama_ucirvine,"['1', '2', '4']","[1, 0, 0]",False,439.014,1.928,5.39014,10.3934,-2.057
2,delaware_alabamaam_ucirvine,"['0', '3', '4']","[0, 1, 0]",False,491.429,1.828,5.91429,10.8125,-1.638
3,delaware_nalabama_uic,"['0', '2', '5']","[0, 0, 1]",False,663,1.597,7.63,12.1842,-0.266


In [14]:
pos = positive_df.reset_index(drop=True).style.applymap(color_positive_green)
pos

,event,index[],result,event_status,odds,bet,mult,payout,profit
0,liu_alabamaam_uic,"['1', '3', '5']","[1, 1, 1]",True,2126,1,22.26,22.26,9.81
1,delaware_alabamaam_uic,"['0', '3', '5']","[0, 1, 1]",False,1160,1,12.6,12.6,0.15
2,liu_nalabama_uic,"['1', '2', '5']","[1, 0, 1]",False,1048.33,1.097,11.4833,12.6,0.15
3,liu_alabamaam_ucirvine,"['1', '3', '4']","[1, 1, 0]",False,944.857,1.206,10.4486,12.6,0.15


In [15]:
sf49ers = MoneyLine(event="parlay", bet_amount=0.01, odds=110)
sf49ers.print_stats()

    event  bet_amount  odds  multiplier  payout
0  parlay        0.01   110         2.1   0.021


In [16]:
pos = MoneyLine(event="pos", bet_amount=1.0, odds=245)
pos.print_stats()

  event  bet_amount  odds  multiplier  payout
0   pos         1.0   245        3.45    3.45


In [17]:
neg = MoneyLine(event="neg", bet_amount=1.0, odds=-305)
neg.print_stats()

  event  bet_amount  odds  multiplier    payout
0   neg         1.0  -305    1.327869  1.327869


In [18]:
neg = MoneyLine(event="neg", bet_amount=1.0, odds=-285)
neg.print_stats()

  event  bet_amount  odds  multiplier    payout
0   neg         1.0  -285    1.350877  1.350877
